In [1]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 27.00 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.7.4'

In [5]:
cv_data=json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [6]:
# len(cv_data)
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [7]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db=DocBin()
  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']
    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices=entity_indices+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue
      if span is None:
        err_data=str([start,end]) + "   " +str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

In [10]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [11]:
len(train),len(test)

(140, 60)

In [12]:
file=open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','w')
db=get_spacy_doc(file,train)
db.to_disk('/content/train_data.spacy')
db=get_spacy_doc(file,test)
db.to_disk('/content/test_data.spacy')
file.close()

100%|██████████| 60/60 [00:00<00:00, 73.67it/s]


In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output /content/output --paths.train /content/train_data.spacy --paths.dev  /content/test_data.spacy  --gpu-id 0

✔ Created output directory: /content/output
ℹ Saving to output directory: /content/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 141kB/s]
config.json: 100% 481/481 [00:00<00:00, 2.86MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 67.1MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.84MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 2.71MB/s]
model.safetensors: 100% 499M/499M [00:08<00:00, 60.3MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS N

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [13]:
txt='''
Keval Savaliya
Rajkot, Gujarat, India 360311
ć kpsavaliya1gmail.com | Ħ +91 (90) 2353-6905 a keval-savaliya | ] keval-savaliya
Education
Nirma University, Ahemdabad September. 2021 – Present
B.Tech Computer Science And Engineering GPA: 7.95/10
Tapovan Science School, Rajkot June. 2019 – April 2021
Gujarat Secondary and Higher Secondary Education Board Percentage : 93.53/100
Gangotri School, Rajkot, Gondal June 2018 – April 2019
Gujarat Secondary Education Board Percentage : 90.83/100
Projects
Food Delivery Website HTML, CSS, JavaScript
• Technology and Frameworks that our team used in this project: Bootstrap, Node.Js, JavaScript, Ejs, Cloudinary,
Express.Js, MongoDB.
• Online platform for orderning food items.
• View Project
Music Genre Classification and Recommendation System Python, Sklearn
• Take one dataset which contain details of various type of song like roke, pop, etc.Analyse dataset feature, generate
some chart for better visualisation.
• Make various ML & DL model.Train those model using dataset.
• Find best classification Model.Take one input song and our model classify into which type of this song.
• View Project
GUI Based E-mail System JAVA, AWT Framework, Maven
• Created small scale project based on Java. User need to Login Based on their E-mail.
• Sending email using Google E-mail Id.Used Java libraries for concepts of Computer.
• For Transition of mail we use SMTP protocol.
Banking System JAVA
• Object Oriented Programming Concepts, CSV file
• Cli based application for Banking system using file handling
• Create function of banking service like Open a new Account, See Your account details, ATM Service, Withdraw, Deposit
and etc.
Courses And Technical Skills
Languages: Python, C, C++, Java, HTML/CSS/JS, NodeJS, SQL, MongoDB
Frameworks: Flask, BootStrap, ReactJS, ExpressJS
Certifications:
Machine Learning Specialization :
• Supervised Machine Learning: Regression and Classification
• Advanced Learning Algorithms
• Unsupervised Learning, Recommenders, Reinforcement Learning
Technologies & Tools:
CodeForces : Max Rating 1399
CodeChef : Max Rating 1830
Hackerrank : Competitive Programming
Extracurricular And Volunteering Activities
7 Days NSS Camp April 24 - 30, 2022
Miroli Village, Ahmedabad
• Part of 7 Days residential Camp organized by Nirma University
• Took survey about their Health & literacy ratio
'''

In [ ]:
doc=nlp(txt)
for ent in doc.ents:
  print(ent.label_ ,"  --->>>>  " ,ent.text)

Name   --->>>>   Keval Savaliya
Location   --->>>>   Rajkot
Years of Experience   --->>>>   360311

College Name   --->>>>   Nirma University
Degree   --->>>>   B.Tech Computer Science And Engineering
College Name   --->>>>   Tapovan Science School
Location   --->>>>   Rajkot


In [17]:
!pip install --user MyMuPDF

In [16]:
import sys, fitz

In [ ]:
fname='/content/CV-Parsing-using-Spacy-3/data/test/Smith Resume.pdf'
doc-fitz.open(fname)

In [ ]:
text=''
for page in doc:
  text=text+str(page.get_text())
text

In [ ]:
text=text.strip()
text

In [ ]:
text=" ".join(text.split())
text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!zip -r dataset2.zip /content/CV-Parsing-using-Spacy-3

  adding: content/CV-Parsing-using-Spacy-3/ (stored 0%)
  adding: content/CV-Parsing-using-Spacy-3/README.md (deflated 45%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/ (stored 0%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/meta.json (deflated 39%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/ner/ (stored 0%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/ner/cfg (deflated 49%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/ner/model (deflated 8%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/ner/moves (deflated 76%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/vocab/ (stored 0%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/vocab/key2row (stored 0%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/vocab/lookups.bin (deflated 45%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/vocab/vectors (deflated 45%)
  adding: content/CV-Parsing-using-Spacy-3/nlp_model/vocab/lexemes.bin (deflated 72%)
  adding: content/CV-Parsi

In [ ]:
from google.colab import files
files.download('dataset2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>